In [1]:
import torch
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

In [4]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((227, 227)),  # Resize images to 32x32 pixels
    torchvision.transforms.ToTensor(),        # Convert images to PyTorch tensors
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))  # Normalize with mean=0.5, std=0.5
])

# Load the MNIST training dataset
# - root: Directory to store/download the dataset
# - train: True for training set
# - download: True to download the dataset if not present
# - transform: Apply the defined transformations
train_dataset = torchvision.datasets.MNIST(
    root='/home/kami/Documents/datasets/',
    train=True,
    download=False,
    transform=transform
)

# Create a DataLoader to batch and shuffle the training data
# - batch_size: 64 for manageable batch processing
# - shuffle: True to randomize the data order
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

# Optional: Load the test dataset with the same transformations
test_dataset = torchvision.datasets.MNIST(
    root='/home/kami/Documents/datasets/',
    train=False,
    download=False,
    transform=transform
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False  # No need to shuffle test data
)

# Example: Iterate through one batch to verify the data
for images, labels in train_loader:
    print(f"Batch shape: {images.shape}")  # Should be [64, 1, 32, 32] (batch, channels, height, width)
    print(f"Labels shape: {labels.shape}")  # Should be [64]
    print(f"Image tensor min: {images.min()}, max: {images.max()}")  # Check normalization
    break  # Only print the first batch




Batch shape: torch.Size([64, 1, 227, 227])
Labels shape: torch.Size([64])
Image tensor min: -1.0, max: 1.0


In [8]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10, in_channels=1):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        # print("size 01:" , x.shape)
        out = self.layer1(x)
        # print("size 02:" , out.shape)
        out = self.layer2(out)
        # print("size 03:" , out.shape)
        out = self.layer3(out)
        # print("size 04:" , out.shape)
        out = self.layer4(out)
        # print("size 05:" , out.shape)
        out = self.layer5(out)
        # print("size 06:" , out.shape)
        out = out.reshape(out.size(0), -1)
        # print("size 07:" , out.shape)
        out = self.fc(out)
        # print("size 08:" , out.shape)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 10
num_epochs = 20
batch_size = 64
learning_rate = 0.005

model = AlexNet(num_classes=num_classes , in_channels=1).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

# Train the model
total_step = len(train_loader)

In [13]:
import time
total_step = len(train_loader)

st = time.time()

model.train()
for epoch in range(10):
    batch_index = 0
    for images, labels in train_loader:
        # Move data to device
        images = images.to(device)
        labels = labels.to(device)

        # Zero out gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Log loss every 100 batches
        batch_index += 1
        if batch_index % 100 == 0:
            print(f"Epoch: {epoch} | Batch: {batch_index} | Loss: {loss.item():.4f}")

et = time.time()
print(et-st)
print("Training completed.")

Epoch: 0 | Batch: 100 | Loss: 0.0346
Epoch: 0 | Batch: 200 | Loss: 0.2987
Epoch: 0 | Batch: 300 | Loss: 0.1761
Epoch: 0 | Batch: 400 | Loss: 0.0652
Epoch: 0 | Batch: 500 | Loss: 0.0708
Epoch: 0 | Batch: 600 | Loss: 0.0252
Epoch: 0 | Batch: 700 | Loss: 0.1092
Epoch: 0 | Batch: 800 | Loss: 0.0090
Epoch: 0 | Batch: 900 | Loss: 0.0094
Epoch: 1 | Batch: 100 | Loss: 0.0573
Epoch: 1 | Batch: 200 | Loss: 0.0107
Epoch: 1 | Batch: 300 | Loss: 0.0183
Epoch: 1 | Batch: 400 | Loss: 0.0312
Epoch: 1 | Batch: 500 | Loss: 0.0232
Epoch: 1 | Batch: 600 | Loss: 0.0202


KeyboardInterrupt: 